In [1]:
import pandas as pd 
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import tree

In [2]:
from auxiliaryFunctions import getMetrics, writeModelResults, readModels, getModelPredictions, get_topModels, ensemblePredictions, ensembleMetrics

Instructions for updating:
non-resource variables are not supported in the long term


In [3]:
data = pd.read_csv('./input/data_titanic_proyecto.csv')
data.head()

,PassengerId,Name,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,passenger_class,passenger_sex,passenger_survived
0,1,"Braund, Mr. Owen Harris",22.0,1,0,A/5 21171,7.2500,NaN,S,Lower,M,N
1,2,"Cumings, Mrs. John Bradley (Florence Briggs Th...",38.0,1,0,PC 17599,71.2833,C85,C,Upper,F,Y
2,3,"Heikkinen, Miss. Laina",26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,Lower,F,Y
3,4,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",35.0,1,0,113803,53.1000,C123,S,Upper,F,Y
4,5,"Allen, Mr. William Henry",35.0,0,0,373450,8.0500,NaN,S,Lower,M,N


In [4]:
results = pd.read_csv('./results/model_results.csv')
results['model_type'] = results['model'].apply(lambda row: row.split('_', 1)[0])
results.head()

,model,accuracy,error,precision,recall,f1-score,model_type
0,decisionTree_criterion=entropy_depth=None_mins...,0.8042,0.1958,0.7358,0.7358,0.7358,decisionTree
1,decisionTree_criterion=entropy_depth=10_minspl...,0.8112,0.1888,0.7708,0.6981,0.7327,decisionTree
2,decisionTree_criterion=gini_depth=10_minsplit=...,0.7972,0.2028,0.7308,0.7170,0.7238,decisionTree
3,SVM_C=1_kernel=rbf__SibSp_Parch_Fare_passenger...,0.7832,0.2168,0.9231,0.4528,0.6076,SVM
4,SVM_C=2_kernel=rbf__SibSp_Parch_Fare_passenger...,0.8252,0.1748,0.8684,0.6226,0.7253,SVM


In [19]:
top_models = get_topModels(results)
top_models

d:\cgarcia\Galileo\2do_Trimestre\StatisticalLearning_I\project\auxiliaryFunctions.py:149: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  top_models = df.groupby('model_type')['accuracy', 'model'].max()


,accuracy,model
model_type,,
NaiveBayes,0.8112,NaiveBayes__SibSp_Parch_Fare_passenger_class_i...
SVM,0.8322,SVM_C=4_kernel=rbf__SibSp_Parch_Fare_passenger...
decisionTree,0.8112,decisionTree_criterion=gini_depth=10_minsplit=...
regLog,0.8112,regLog_lr=0.1_reg=l2_lambda=0.0001_mbatch=20_e...


In [6]:
d = {'SibSp': [0.2, 0.1]
    ,'Parch':	[0.166667, 0.15]
    ,'Fare': [0.044979, 0.032]	
    ,'passenger_class': [0, 1]
    ,'isFemale': [1, 1]
}

X_test = pd.DataFrame(data = d)
X_test

,SibSp,Parch,Fare,passenger_class,isFemale
0,0.2,0.166667,0.044979,0,1
1,0.1,0.150000,0.032000,1,1


In [ ]:
def simulatePrediction(x_predict):
    predictions_df = ensemblePredictions(top_models, X_test)
    mode_pred = predictions_df['mode'].values

    pred_tree = tree.plot_tree(model, filled = True)
    #plt.show()

In [9]:
predictions_df = ensemblePredictions(top_models, X_test)
predictions_df.head()

,NaiveBayes,SVM,decisionTree,regLog,mode
0,1,0,1,1,1
1,1,1,0,1,1


array([[0.94497707, 1.14870139],
       [0.18105566, 3.30475271]])

In [23]:
model = readModels(top_models.loc['regLog', 'model'], 'regLog')
getModelPredictions(model, X_test, 'regLog', True)

array([0.59900516, 0.9191816 ], dtype=float32)

In [10]:
#tree.plot_tree(model, filled = True)
#plt.show()

In [14]:
model_metrics = ensembleMetrics(predictions_df, [0, 1])
model_metrics

,model,accuracy,error,precision,recall,f1-score
0,NaiveBayes,0.5,0.5,0.5,1.0,0.6667
1,SVM,1.0,0.0,1.0,1.0,1.0000
2,decisionTree,0.0,1.0,0.0,0.0,0.0000
3,regLog,0.5,0.5,0.5,1.0,0.6667
4,mode,0.5,0.5,0.5,1.0,0.6667
